First try for creating likelihood plots. Some code snipplets are copied from https://github.com/ElisabethRoesch/Bifurcations 

###### [1] Roesch, E., & Stumpf, M. P. H. (2019). Parameter inference in dynamical systems with co-dimension 1 bifurcations. *Royal Society Open Science, 6*(190747). [https://doi.org/10.1098/rsos.190747](https://royalsocietypublishing.org/doi/pdf/10.1098/rsos.190747)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
import numpy as np

from numpy import genfromtxt
from matplotlib import ticker, cm
from matplotlib.patches import Rectangle
from numpy import inf
from numpy import NaN

In [ ]:
#LL info
alphas = [5] # corresponds to number of parameter a (start counting from 1)
ics = [6] # corresponds to number of n0
noises = [1] # number of noise
fig = plt.figure(figsize=(15,5))

In [ ]:
# help functions

def get_ll_alps_one(alp_ind): # converts number of alpha to actual alpha value (alp_ind is number of alpha value)
    #print('alp_ind',alp_ind)
    alp = [-1.5,-0.8,-0.5,-0.2, 0, 0.2,0.5,0.8,1.5] # actual alpha values they have data for
    real_value = alp[alp_ind-1]
    #print('real_value',real_value)
    ratio_value = abs((-2)-(real_value))/4
    #print('ratio_value',ratio_value)
    new_value = (ratio_value)*10
    #print('new_value',new_value)
    return new_value

def get_ll_ics_one(ics_ind): # converts number of ic to actual ic value
    #print('ics_ind',ics_ind)
    ics= [-1,-0.6,-0.4,-0.1,0,0.1,0.4,0.6,1] # actual ic values they have data for 
    real_value = ics[ics_ind-1]
    #print('real_value',real_value)
    ratio_value = abs((-2)-(real_value))/4
    #print('ratio_value',ratio_value)
    new_value = (ratio_value)*10
    #print('new_value',new_value)
    return new_value 

def read_ll_file(alpha,init,bifurcation_type,noise,m): # alpha, ic, input is number of alpha / ic value
    path_to_file= "/home/elisabeth/master_data/er4517/public_html/assets/js/ll/"        #TODO
    name=str(bifurcation_type)+"_"+str(m)+"_"+str(noise)+"_"+str(alpha)+"_"+str(init)   #TODO
    ending=".csv"
    csv = np.genfromtxt (path_to_file+name+ending, skip_header=1, delimiter=",")
    #csv2=np.nan_to_num(csv)
    #print(csv2)
    #THINK ABOUT THIS LINE!!!!
    csv[csv == -inf] = -1000
    return csv

def make_ll_plot(ax,csv,ind): # run this func for all prm combinations to consider (all ind to consider). ind is index of alphas array (which contains the numbers of alphas)
    #countouring=ax.contourf(csv,30,cmap='Reds', alpha=0.9,linecolor='white',vmin=-1000, vmax=0)
    al_plot_nr = get_ll_alps_one(alphas[ind])
    ic_plot_nr = get_ll_ics_one(ics[ind])
    
    alpha_x_ticks = [-2,-1,0,1,2]
    alpha_x_ticks_loc = [0,2.5,5,7.5,10]
    inits_y_ticks = alpha_x_ticks
    alpha_y_ticks_loc = alpha_x_ticks_loc
    texts=["a","b","c","d","e"]
    
    plt.annotate(texts[ind], (al_plot_nr+0.7, ic_plot_nr+0.7) ,size=20,color='black')
    plt.plot([al_plot_nr],[ic_plot_nr],marker="x",label="Truth",color="white",markersize=16,markerfacecolor='gold',markeredgewidth=1.5, markeredgecolor="gold")
    if (np.isfinite(csv).any()):   
        ax.grid(color='grey', linestyle='-',alpha=0.1, linewidth=1)
        ax.set_facecolor('white')
        #print(csv)
        countouring=ax.contourf(csv,30,cmap='Reds',alpha=0.9,vmin=-1000, vmax=0)
        plt.xticks(alpha_x_ticks_loc,alpha_x_ticks)
        plt.yticks(alpha_y_ticks_loc,inits_y_ticks)
        plt.xlabel("α")
        plt.ylabel("IC")
        #plt.legend(loc='upper left')
        cbar = fig.colorbar(countouring,fraction=0.09)
        cbar.ax.set_ylabel('Log-Likelihood')
        #cbar.ax.set_yticklabels(['High',  'Low'], rotation=90) 
        return countouring
    else:      
        print("eieiei")
        
        csv[csv == -inf] = -1000
        #print(csv)
        countouring=ax.contourf(csv,39,cmap='Reds',alpha=0.9,vmin=-1000, vmax=0)
        plt.xticks(alpha_x_ticks_loc,alpha_x_ticks)
        plt.yticks(alpha_y_ticks_loc,inits_y_ticks)
        plt.xlabel("α")
        plt.ylabel("IC")
        cbar = fig.colorbar(countouring,fraction=0.09)
        cbar.ax.set_ylabel('Log-Likelihood')
        return("eieieie")